In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Activation, BatchNormalization, Concatenate, Dense, Embedding, Flatten, Reshape, Input, Multiply,Concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv1D
from keras.models import Sequential, Model
from keras.optimizers import Adam

In [2]:
import math

In [3]:
from gensim.models import word2vec
import MeCab
import json
import hashlib
from googletrans import Translator
from keras.layers import Dropout
from keras.utils import np_utils
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import random
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import torch
from transformers.modeling_bert import BertModel
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer
%matplotlib inline

In [4]:
translator = Translator()
mt = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/')
mt.parse('')
tokenizer = BertJapaneseTokenizer.from_pretrained('bert-base-japanese-whole-word-masking')
model_bert = BertModel.from_pretrained('bert-base-japanese-whole-word-masking')

In [5]:
def get_vector_cls(text):
    input_ids = tokenizer.encode(text, return_tensors='pt') 
    result = model_bert(input_ids)
    tensor_result = result[0][0][0]
    numpy_result = tensor_result.to('cpu').detach().numpy().copy()
    return numpy_result

In [6]:
class Event:
    def __init__(self, id, type, score, desc, links):
        self.id = id
        self.type = type
        self.score = score
        self.desc = desc
        self.links = links

In [7]:
# JSON ファイルから event set をロード
def load_events(jsonfile):
    with open(jsonfile) as f:
        df = json.load(f)
    events = {x['id']: Event(x['id'], x['type'], x['score'], x['desc'], x['links']) for x in df} #eventsにidをkeyとしそのオブジェクトをvalueとした辞書を生成
    for k,x in events.items():
        x.links = [events[e] for e in x.links] #Event.linkの中身をidの配列からEventの配列に変更
    return events

In [8]:
events = load_events('sesaku2.json')

In [10]:
import pandas as pd
data = []
labels = []
columns=[]
index=[]
for k1, v1 in events.items():
    if v1.type[-1]=='部品':
        index.append(v1.desc)
for k1, v1 in events.items():
    if v1.type[-1]=='対策':
        if not v1.desc in columns:
            columns.append(v1.desc)
df = pd.DataFrame(index=index, columns=columns)
for k1, v1 in events.items():
    if v1.type[-1]=='部品':
        for k2, v2 in events.items():
            if v2.type[-1] == '対策':
                    if v2 in v1.links:
                        df.at[v1.desc, v2.desc] = 1
                    else:
                        df.at[v1.desc, v2.desc] = 0

In [11]:
df.shape

(173, 319)

In [15]:
taisaku_vec_word = {}
taisaku_count ={}
for i in df:
    taisaku_vec_word[i]=get_vector_cls(i)
    taisaku_count[i] = 0

#label →部品　生成→対策
data = []
buhin = []
buhin2 = []
taisaku = []
buhin_dict = {}
# data 
for index, row in df.iterrows():#部品
    print('さ')
    x1 = get_vector_cls(index)
    buhin.append(x1)
    buhin_dict[index] = tuple(x1)
    for i in df:#対策
        x2 =  taisaku_vec_word[i]#対策
        if row[i] ==1:
            data.extend([np.append(x1, x2)])
            buhin2.append(x1)
            taisaku_count[i] += 1
            
for k,v in taisaku_vec_word.items():
    taisaku.append(v)
data = np.array(data)
buhin = np.array(buhin)

さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ
さ


In [16]:
#関係がある部品と対策の文書ベクトル
data = np.array(data)
data.shape

(7303, 1536)

In [17]:
buhim = []
for i in data:
    buhim.append(i[:768])
buhim  = np.array(buhim)

In [18]:
buhim.shape

(7303, 768)

In [19]:
#部品の文書ベクトルだけ
buhin = np.array(buhin)
buhin.shape

(173, 768)

In [20]:
buhin2 = np.array(buhin2)
buhin2.shape

(7303, 768)

In [21]:
buhin.min()

-1.473694

In [22]:
#対策の文書ベクトルだけ
taisaku = np.array(taisaku)
taisaku.shape

(319, 768)

In [23]:
taisaku_map = sorted(taisaku_count.items(),key=lambda x:x[1], reverse=True)
taisaku_map

[('多能工化を設計者に適用。メカ・電気・ソフトの各スキルマップを作成し統合。', 152),
 ('定性的なアウトプットである個人の振り返りを実施。個人の成長・マネジメント軸を強化する。', 152),
 ('６ヶ月に一回のサイクルで実施。', 152),
 ('会社目標から製品、各機能組織ごとの、目標展開を実施。メンバー目標とも連携している', 152),
 ('背景・目的・目標・KGI/KPI・大日程、現状の課題までが繋がっている', 152),
 ('関連部門が参加した課題解決の仕組みがあり、PLMを使ったデジタル化によるグローバル会議ができる', 152),
 ('機能と役割分担表があり、リーダとサブリーダが決まっている。', 152),
 ('PLMを使ったグローバルリアルタイムコラボレーション会議を実施。', 152),
 ('5M (人・道具・原料・方法・測定・環境)の基準で変化点を定義し、最新の状態を関係者で共有している', 152),
 ('5M変動時の作業指導、品質確認等ができている', 152),
 ('製造部門だけでなく開発・設計などホワイトカラー業務にも適用できれいる。', 152),
 ('モジュール化の定義、進め方が決められており、原価まで繋がった評価ができる', 152),
 ('コンフィグレータが、製品構成、評価、生産まで考慮した仕様になっている。', 152),
 ('KGI/KPIに標準化の指標がある', 152),
 ('製品開発ゲートの定義と、フェーズごとのアプトプットが明確になっている', 152),
 ('先行開発ゲートの定義は、通常開発の定義と異なっており、技術の棚という考え方がある', 152),
 ('過去の作業マニュアルが適切に更新され、処分されている', 152),
 ('製品・部品の品質管理が定常的に実施されており、管理図などによりモニタリングしている', 152),
 ('デザインレビューのパフォーマンスを測定できる指標がある。', 152),
 ('グローバルに関連部門が一元化した製品情報を使って実施できる。', 152),
 ('DR結果を一元化したE-BOMに蓄積して、再利用できる。', 152),
 ('施策は、複数の候補を用意し、各効果を定量的に測れる、', 152),
 ('工程設計をの５点満

# GANの生成

In [39]:
#パラメータ
vec_length = 768
vec_shape =(vec_length,)
z_dim = 50

In [40]:
 #生成器
def build_generator(z_dim,vec_shape):
    
        model = Sequential()
        model.add(Dense(128,input_dim =( z_dim+vec_shape[0])))
        #model.add(BatchNormalization())
        model.add(Activation('relu'))
 
        model.add(Dense(768))
        model.add(Activation('tanh'))
        print('generator')
        model.summary()
        return model

In [41]:
#識別器
def build_discriminator(vec_shape):
    
    model = Sequential()
    
    model.add(Dense(128, input_dim=vec_shape[0]*2)) #activation
    model.add(Activation('relu'))
    
    model.add(Dropout(0.2))
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    print('discriminator')
    model.summary()
    return model

In [42]:
#generatorを学習する時のもの
def build_combined(generator,discriminator):
    z_y = Input(shape=(z_dim+vec_shape[0],)) #ランダムベクトル＋部品ベクトル
    vec_1 = Input(shape=vec_shape)              #部品ベクトル
    
    #ベクトルを生成
    gen_vec = generator(z_y)
    
    #discriminatorに入力するやつ
    vec_2 = Concatenate()([vec_1,gen_vec])  #[部品ベクトル,対策ベクトル]
    #discriminatorの重みとバイアスを固定
    discriminator.trainable = False
    
    valid = discriminator(vec_2)

    model = Model(inputs = [z_y, vec_1], outputs = valid)
    
    print('combined')
    model.summary()
    return model

In [109]:
#コンパイル
discriminator = build_discriminator(vec_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(),metrics=['accuracy'])

generator = build_generator(z_dim,vec_shape)
    
combined = build_combined(generator,discriminator)
    
combined.compile(loss='binary_crossentropy',optimizer=Adam())

discriminator
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 128)               196736    
_________________________________________________________________
activation_19 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)               16512     
_________________________________________________________________
activation_20 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 1)  

In [110]:
def data_scaling(data,change_min,change_max):
    max_data = 10 #実数より大きい整数
    min_data = -5#実数より小さい整数
    data_std = (data - min_data) / (max_data - min_data)
    data = data_std*(change_max - change_min) + change_min
    return data , min_data, max_data

In [111]:
def data_scaling_inverse(data,change_min,change_max):
    max_data = 1 #実数より大きい整数
    min_data = -1#実数より小さい整数
    data_std = (data - min_data) / (max_data - min_data)
    data = data_std*(change_max - change_min) + change_min
    return data , min_data, max_data

In [112]:
losses = []
accuracies = []
iteration_checkpoints = []
def train(discriminator_input, iterations, batch_size, sample_interval):
    
    generator_input = []
    for i in discriminator_input:
        generator_input.append(i[:768])
    generator_input  = np.array(generator_input)
    
    #generator_input = 部品ベクトル
    #discriminator_input = [部品ベクトル,対策ベクトル]
    #-1~1へスケーリング
    generator_input , _ , _ = data_scaling(generator_input,-1,1)
    discriminator_input , _ , _ =  data_scaling(discriminator_input,-1,1)
    
    
    #ラベル1
    real = np.ones((batch_size,1))
    #ラベル0
    fake = np.zeros((batch_size,1))
    
    for iteration in range(iterations):
        
        #-------------------
        #識別器の訓練
        #-------------------
        
        #本物のデータから適当に選ぶ
        idx = np.random.randint(0,len(discriminator_input),batch_size)
        #discriminatorの入力にいれる本物の[部品ベクトル,対策ベクトル]
        real_discriminator_input = discriminator_input[idx]
        
        #部品のベクトルの情報を含んだランダムベクトルを作る
        
        #noize_y = np.random.randint(0,len(generator_input),batch_size)
        
        noize_y = generator_input[idx]
        
        #平均0標準偏差１の正規分布に従う乱数を生成
        noize_z = np.random.normal(0,2,(batch_size, z_dim))
        
        #ランダムベクトルと部品のベクトルを結合
        noize_z_y = np.concatenate((noize_z, noize_y),axis=1)
        
        #ジェネレータで偽の対策文書ベクトルを生成させる
        gen_vec = generator.predict(noize_z_y)
        
        #if (iteration +1) % sample_interval == 0:
            #print(gen_vec)
            #print(noize_z)
        #discriminatorの中にいれる偽の[部品ベクトル,対策ベクル]の生成
        fake_discriminator_input = np.concatenate((noize_y,gen_vec),axis=1)
        
        d_loss_real = discriminator.train_on_batch(real_discriminator_input, real)
        d_loss_fake = discriminator.train_on_batch(fake_discriminator_input, fake)
        
        #それぞれの損失関数を平均
        d_loss, accuracy = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        #-------------------
        #生成器の訓練
        #-------------------
        #部品のベクトルの情報を含んだランダムベクトルを作る
        
        noize_y = np.random.randint(0,len(generator_input),batch_size)
        
        noize_y = generator_input[noize_y]

        #平均0標準偏差１の正規分布に従う乱数を生成
        noize_z = np.random.normal(0,2,(batch_size, z_dim))

        #ランダムベクトルと部品のベクトルを結合
        noize_z_y = np.concatenate((noize_z, noize_y),axis=1)

        #ジェネレータで偽の対策文書ベクトルを生成させる
        gen_vec = generator.predict(noize_z_y)
        
        g_loss = combined.train_on_batch([noize_z_y,noize_y],real)
        
        if (iteration +1) % sample_interval == 0:
            losses.append((d_loss, g_loss))
            accuracies.append(100.0 * accuracy)
            iteration_checkpoints.append(iteration +1)
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" %(iteration +1,d_loss, 100.0*accuracy,g_loss))
    return generator,  discriminator

In [113]:
train_data, test_data = train_test_split(data)

In [114]:
train_data = np.array(train_data)

In [141]:
iterations =2000
batch_size = 32
sample_interval = 1000

generator , discriminator= train(data , iterations, batch_size,sample_interval)

1000 [D loss: 0.000000, acc.: 100.00%] [G loss: 86.892822]
2000 [D loss: 0.000000, acc.: 100.00%] [G loss: 105.164200]


In [142]:
buhin_name_list = []
for index, row in df.iterrows():#部品
    buhin_name_list.append(index)

In [143]:
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
class pycolor:
    RED = '\033[31m'
    BLUE = '\033[34m'
    END = '\033[0m'

In [144]:
def search_taisaku(text):
    vec = weighted_mean_vec(text)
    scaling_vec ,max_data , min_data = data_scaling(vec,-1,1)
    print(max_data,min_data)
    scaling_vec = scaling_vec.reshape(1,768)
    noize_z = np.random.normal(0,1,(1, z_dim))
    noize = np.concatenate((noize_z, scaling_vec),axis=1)
    gen = np.array(generator(noize))
    result , _ , _ = data_scaling(gen, -5, 10) 
    score_sim = {}
    for k,v in taisaku_vec_word.items():
        score_sim[k]=cos_sim(result, v)
    score_sort = sorted(score_sim.items(),key=lambda x:x[1], reverse=True)
    for i_1, i_2 in score_sort[0:10]:
        print(i_1)
        print(i_2)

In [145]:
buhin_name_list = []
for index, row in df.iterrows():#部品
    buhin_name_list.append(index)

In [146]:
def test(text):
    print('---------------部品----------------')
    print(text)
    print('---------------対策----------------')
    taisaku = []
    for index, row in df.iterrows():#部品
            if (index == text):
                for i in df:#対策
                    if row[i] ==1:
                        #print(i)
                        taisaku.append(i)
                break
    print('---------------予想----------------')
    vec = get_vector_cls(text)
    scaling_vec , max_data , min_data = data_scaling(vec,-1,1)
    scaling_vec = scaling_vec.reshape(1,768)
    noize_z = np.random.normal(0,1,(1, z_dim))
    noize = np.concatenate((noize_z, scaling_vec),axis=1)
    gen = np.array(generator(noize))
    result , _ , _ = data_scaling_inverse(gen, -5, 5)
    ####print(result)
    score_sim = {}
    for k,v in taisaku_vec_word.items():
        score_sim[k]=cos_sim(result, v)
    score_sort = sorted(score_sim.items(),key=lambda x:x[1], reverse=True)
    for i_1, i_2 in score_sort[0:10]:
        print(i_1)
        print(i_2)
        print(taisaku_count[i_1])
        if i_1 in taisaku:
            print(pycolor.BLUE+"ある"+pycolor.END)
        else:
            print(pycolor.RED+"なし"+pycolor.END)

In [147]:
text_1 = buhin_name_list[0]
test(text_1)

---------------部品----------------
エンジン搭載 吸気レゾネーター
---------------対策----------------
---------------予想----------------
DR結果を一元化したE-BOMに蓄積して、再利用できる。
[0.03591824]
152
ある
品名単位でものと情報の流れが見える化。リードタイム短縮施策があり実施。
[0.0250303]
0
なし
職務拡充により作業者のモチベーション向上。品質を作りこむ意識が醸成。
[0.02337536]
152
ある
ATシフトケーブルのクランプ最適化。PCU座面設定。
[0.02278919]
5
なし
メインインインタミシャフト圧縮、ストロークの確認。
[0.02155635]
5
なし
ロアアーム、ケージダウン検討
[0.01963898]
0
なし
ストラットの共通化。効果を明確に。
[0.01635489]
0
なし
一元化されたEBOM/PBOM/MBOMがありデータ連携
[0.016139]
0
なし
CAE解析データを作成し衝突ストーリーを作成する
[0.01598884]
21
なし
マルチOEM管理を実施、各BOMをクラウドを使って実施できる。
[0.01592424]
21
なし


text_1 = buhin_name_list[10]
test(text_1)

In [148]:
text_1 = buhin_name_list[50]
test(text_1)

---------------部品----------------
エンジン本体 オイルパン　サブアセンブリ
---------------対策----------------
---------------予想----------------
DR結果を一元化したE-BOMに蓄積して、再利用できる。
[0.03591824]
152
なし
品名単位でものと情報の流れが見える化。リードタイム短縮施策があり実施。
[0.0250303]
0
なし
職務拡充により作業者のモチベーション向上。品質を作りこむ意識が醸成。
[0.02337536]
152
なし
ATシフトケーブルのクランプ最適化。PCU座面設定。
[0.02278919]
5
なし
メインインインタミシャフト圧縮、ストロークの確認。
[0.02155635]
5
なし
ロアアーム、ケージダウン検討
[0.01963898]
0
なし
ストラットの共通化。効果を明確に。
[0.01635489]
0
なし
一元化されたEBOM/PBOM/MBOMがありデータ連携
[0.016139]
0
なし
CAE解析データを作成し衝突ストーリーを作成する
[0.01598884]
21
ある
マルチOEM管理を実施、各BOMをクラウドを使って実施できる。
[0.01592424]
21
ある


In [149]:
def test(text):
    taisaku = []
    precition = 0
    for index, row in df.iterrows():#部品
            if (index == text):
                for i in df:#対策
                    if row[i] ==1:
                        #print(i)
                        taisaku.append(i)
                break
    vec = get_vector_cls(text)
    scaling_vec , max_data , min_data = data_scaling(vec,-1,1)
    scaling_vec = scaling_vec.reshape(1,768)
    noize_z = np.random.normal(0,1,(1, z_dim))
    noize = np.concatenate((noize_z, scaling_vec),axis=1)
    gen = np.array(generator(noize))
    result , _ , _ = data_scaling_inverse(gen, -5, 5)
    score_sim = {}
    for k,v in taisaku_vec_word.items():
        score_sim[k]=cos_sim(result, v)
    score_sort = sorted(score_sim.items(),key=lambda x:x[1], reverse=True)
    for i_1, i_2 in score_sort[0:10]:
        if i_1 in taisaku:
            precition += 1
    return precition/10

In [150]:
total_precition = 0
tt = 0
for ei in range(5):
    total_precition = 0
    for i in range(173):
        text_1 = buhin_name_list[i]
        precition = test(text_1)
        total_precition += precition
    print(total_precition/173)
    tt += total_precition/173
tt/5

0.2057803468208091
0.2057803468208091
0.2057803468208091
0.2057803468208091
0.2057803468208091


0.2057803468208091

In [151]:
buhim = []
test_buhin_name = []
for i in test_data:
    buhim.append(tuple(i[:768]))
for k,v in buhin_dict.items():
    if v in buhim:
        test_buhin_name.append(k)

In [ ]:
total_precition = 0
tt = 0
for ei in range(5):
    total_precition = 0
    for i in range(len(test_buhin_name)):
        text_1 =test_buhin_name[i]
        precition = test(text_1)
        total_precition += precition
    print(total_precition/len(test_buhin_name))
    tt += total_precition/173
tt/5

In [127]:
# 1000  21.1
# 2000  20.5
# 5000  20.5
#10000  21.7

In [ ]:
a = [21.1,20.5,20.5,21.7]
sum(a)/4

# a

In [128]:
text_1 = buhin_name_list[100]
test(text_1)

0.2

##  これはモード崩壊（mode collapse）と呼ばれる現象で、generatorの学習に失敗して、訓練データの（しばしば多峰性の）分布全体を表現できずに訓練データの最頻値（mode）のみを学習してしまいます。全国民の期待に応える能力がなく、とりあえず多数派のための政策をつくる、みたいなイメージですかね。

In [178]:
generator_input , _ , _ = data_scaling(buhin,-1,1)
discriminator_input , _ , _ =  data_scaling(data,-1,1)

In [135]:
noize_y = np.random.randint(0,len(buhin),batch_size)
noize_y = buhin2[noize_y]
noize_z = np.random.normal(0,1,(batch_size, z_dim))
noize_z_y = np.concatenate((noize_z, noize_y),axis=1)
gen_vec = generator.predict(noize_z_y)

In [139]:
gen_vec

array([[ 0.4359803 , -0.05961279, -0.04837259, ...,  0.00045572,
         0.2699624 ,  0.04629831],
       [ 0.4359803 , -0.05961279, -0.04837259, ...,  0.00045572,
         0.2699624 ,  0.04629831],
       [ 0.4359803 , -0.05961279, -0.04837259, ...,  0.00045572,
         0.2699624 ,  0.04629831],
       ...,
       [ 0.4359803 , -0.05961279, -0.04837259, ...,  0.00045572,
         0.2699624 ,  0.04629831],
       [ 0.4359803 , -0.05961279, -0.04837259, ...,  0.00045572,
         0.2699624 ,  0.04629831],
       [ 0.4359803 , -0.05961279, -0.04837259, ...,  0.00045572,
         0.2699624 ,  0.04629831]], dtype=float32)

In [140]:
noize_z_y

array([[-0.35201003, -0.48215373, -0.3709283 , ..., -0.37396668,
         1.54047585, -0.89240008],
       [ 1.02011903, -0.99039578,  0.6147865 , ..., -0.22950753,
         1.30674742, -1.04674338],
       [-1.89017538,  0.45027579, -0.33180167, ..., -0.37396668,
         1.54047585, -0.89240008],
       ...,
       [ 1.13324786,  0.56365652, -0.66692741, ..., -0.49368474,
         1.28149669, -0.86522065],
       [ 0.68343362, -2.26604594, -0.14852601, ..., -0.49368474,
         1.28149669, -0.86522065],
       [ 0.47330225, -1.55339981, -0.02546597, ..., -0.49368474,
         1.28149669, -0.86522065]])

In [141]:
noize_y

array([[ 1.38220024,  0.61715281, -0.45223216, ..., -0.37396668,
         1.54047585, -0.89240008],
       [ 0.55496857,  1.33560361,  0.10814531, ..., -0.22950753,
         1.30674742, -1.04674338],
       [ 1.38220024,  0.61715281, -0.45223216, ..., -0.37396668,
         1.54047585, -0.89240008],
       ...,
       [ 1.50043327,  0.71683983,  0.20033428, ..., -0.49368474,
         1.28149669, -0.86522065],
       [ 1.50043327,  0.71683983,  0.20033428, ..., -0.49368474,
         1.28149669, -0.86522065],
       [ 1.50043327,  0.71683983,  0.20033428, ..., -0.49368474,
         1.28149669, -0.86522065]])